# LaRA 2 benchmark tutorial

This document assists you to run the benchmarks for LaRA 2 that we describe in our paper.

## Requirements and paths
In order to run all the benchmarks, the following programs and libraries need to be built on your system. Afterwards please specify the paths to the binaries below, if they are not in your PATH.

Program        | Link to the download and install instructions
:------------- |:---------------------------------------------
LaRA 2         | https://seqan.github.io/lara
T-Coffee       | http://www.tcoffee.org/Projects/tcoffee/index.html
MAFFT for LaRA | https://github.com/bioinformatics-polito/LaRA2-mafft
RNAz           | https://www.tbi.univie.ac.at/software/RNAz
SQUID          | http://www.eddylab.org/software.html

Furthermore we use the following Python modules, which can be installed via pip:
```commandline
pip install biopython
pip install wget
```


In [ ]:
LARA_BIN = 'lara'
TCOFFEE_BIN = 't_coffee'
MAFFT_BIN = 'mafft-xinsi'
RNAZ_BIN = 'RNAz'
COMPALIGN_BIN = 'compalignp' # part of the SQUID library
ALISTAT_BIN = 'alistat'      # part of the SQUID library

Let's create a working directory, where we store all the files that we create during the benchmarks.

In [ ]:
import os

WORK_DIR = './benchmark/'
os.system('rm -rf ' + WORK_DIR) # make sure it does not exist
os.makedirs(WORK_DIR)           # create
os.chdir(WORK_DIR)              # use as working directory

## BRAliBase II
We use the data set 1 of [BRAliBase II](http://projects.binf.ku.dk/pgardner/bralibase/bralibase2.html) to observe the performance across several RNA families. 

Download and extract the data:

In [ ]:
import wget
import tarfile

file = wget.download('http://projects.binf.ku.dk/pgardner/bralibase/data-set1.tar.gz')
with tarfile.open(file, 'r:gz') as tar:
    tar.extractall()

Create the list of input files.

In [ ]:
unaligned = []
for path, _, fnames in os.walk('./data-set1/'):
    unaligned.extend([os.path.join(path, f) for f in fnames if f.endswith('.fa') and path.endswith('unaligned')])
unaligned.sort(key=str.lower)

# generate a unique name for output files
def get_name(filename):
    parts = os.path.splitext(filename)[0].split('/')
    return(parts[-3] + '_' + parts[-1])

print('Found ' + str(len(unaligned)) + ' files.')

Execute LaRA 2 with T-Coffee.

In [ ]:
from time import time
from subprocess import run

os.mkdir('lara_tcoffee')
t = time()
for infile in unaligned:
    outfile = 'lara_tcoffee/' + get_name(infile) + '.aln'
    run([LARA_BIN, '-i', infile, '-w', 'pairwise.lib', '-j', '4'], check=True)
    run([TCOFFEE_BIN, '-lib', 'pairwise.lib', '-outfile', outfile], check=True)
    
print('Time: ' + str(time()-t) + ' seconds.')

Evaluation of the Sum of Pair Scores (SPS) and Structure Conservation Index (SCI).

In [ ]:
from Bio import AlignIO
from re import search

stat_file = open('lara_tcoffee.dat', 'w')
print('alignment_id\t%_id\tSPS\tSCI')

for infile in unaligned:
    alid = get_name(infile)
    ref_file = infile.replace('/unaligned/', '/structural/')
    aln_file = 'lara_tcoffee/' + alid + '.aln'
    fa_file = 'lara_tcoffee/' + alid + '.fa'
    
    # convert T-Coffee output to FastA
    ali = AlignIO.read(aln_file, 'clustal')
    AlignIO.write([ali], fa_file, 'fasta')
    
    # get average sequence identity
    out = run([ALISTAT_BIN, '-q', ref_file], capture_output=True, text=True, check=True).stdout
    avg_id = search('(?<=Average identity:)(\s+)(\d+)(?=%)', out).group(2)
    
    # get SPS
    out = run([COMPALIGN_BIN, '-r', ref_file, '-t', fa_file], capture_output=True, text=True, check=True).stdout
    sps = out.strip()
    
    # get SCI
    out = run([RNAZ_BIN, aln_file], capture_output=True, text=True, check=True).stdout
    sci = search('(?<=Structure conservation index:)(\s+)(\d+.\d+)', out).group(2)
    
    # output
    print(alid + '\t' + avg_id + '\t' + sps + '\t' + sci)
    stat_file.write(alid + '\t' + avg_id + '\t' + sps + '\t' + sci + '\n')
    
stat_file.close()

The output is stored in `lara_tcoffee.dat` and can be plotted e.g. with R.
